# DCRouter - Inference

This notebook demonstrates how to use a trained **DCRouter** for inference.

## 1. Environment Setup

In [ ]:
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path(os.getcwd()).parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

os.chdir(PROJECT_ROOT)

In [ ]:
import torch
from llmrouter.models.dcrouter import DCRouter
from llmrouter.utils import setup_environment
import yaml

setup_environment()

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

## 2. Load Trained Router

In [ ]:
CONFIG_PATH = "configs/model_config_train/dcrouter.yaml"

with open(CONFIG_PATH, 'r') as f:
    config = yaml.safe_load(f)

config['model_path']['load_model_path'] = config['model_path']['save_model_path']

INFERENCE_CONFIG_PATH = "configs/model_config_test/dcrouter_inference.yaml"
os.makedirs(os.path.dirname(INFERENCE_CONFIG_PATH), exist_ok=True)

with open(INFERENCE_CONFIG_PATH, 'w') as f:
    yaml.dump(config, f)

router = DCRouter(yaml_path=INFERENCE_CONFIG_PATH)
print(f"Router loaded with {len(router.llm_data)} LLM candidates")

## 3. Query Routing

In [ ]:
EXAMPLE_QUERIES = [
    {"query": "What is the capital of France?"},
    {"query": "Solve the equation: 2x + 5 = 15"},
    {"query": "Write a Python function to check if a number is prime."},
    {"query": "Explain quantum computing in simple terms."},
    {"query": "What are the implications of climate change on agriculture?"},
]

print("Routing Results:")
print("=" * 70)

for i, query in enumerate(EXAMPLE_QUERIES, 1):
    result = router.route_single(query)
    print(f"{i}. {query['query'][:55]}...")
    print(f"   Routed to: {result['model_name']}")

## 4. Batch Routing

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt

# Route multiple queries
results = [router.route_single(q) for q in EXAMPLE_QUERIES]

# Show distribution
model_counts = Counter(r['model_name'] for r in results)

plt.figure(figsize=(10, 5))
plt.bar(model_counts.keys(), model_counts.values())
plt.xlabel('LLM Model')
plt.ylabel('Count')
plt.title('DCRouter Routing Distribution')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:
1. Loading a trained DCRouter
2. Routing queries using transformer-based embeddings
3. Batch routing and visualization

DCRouter is effective for:
- High-accuracy routing with transformer embeddings
- Distinguishing subtle differences between queries